This notebook will be implementing a neural network from scratch in attempts to classify MNIST data correctly. 
We will split the `60,000 training` images into `50,000 training` and `10,000 validation` images, then test it on the `10,000 testing` images.

In [1]:
import numpy as np
import random
import pickle
import mnist_loader

In [3]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [115]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes) # sizes will be a two dimensional array where rows will be layers
        self.sizes = sizes # and each column has only one number indicating neurons in that layer
        self.biases = [np.random.randn(y,1) for y in sizes[1:]] # for every layer, generate random biases pertaining to number of neurons in layer
        self.weights = [np.random.randn(y,x)
                        for x,y in zip(sizes[:-1], sizes[1:])] # make weights that represent connections from one layer to another.   
    
    def feedforward(self, a):
        """Return the output of the network if "a" is input"""
        for b, w in zip(self.biases,self.weights): # we iterate through all the weights and biases 
            a = sigmoid(np.dot(w,a)+b) # we multiply the inputs with all of the weights and standardize by adding the bias for each neuron
        return a
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        
        training_data = list(training_data)
        n = len(training_data)
        
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range (0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feed-forward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives partial C_x 
        partial a for the output activations."""
        return (output_activations-y)

In [116]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [117]:
net = Network([784, 30, 10])

In [118]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8304 / 10000
Epoch 1: 9222 / 10000
Epoch 2: 9338 / 10000
Epoch 3: 9358 / 10000
Epoch 4: 9411 / 10000
Epoch 5: 9421 / 10000
Epoch 6: 9430 / 10000
Epoch 7: 9479 / 10000
Epoch 8: 9457 / 10000
Epoch 9: 9452 / 10000
Epoch 10: 9444 / 10000
Epoch 11: 9475 / 10000
Epoch 12: 9484 / 10000
Epoch 13: 9490 / 10000
Epoch 14: 9488 / 10000
Epoch 15: 9452 / 10000
Epoch 16: 9473 / 10000
Epoch 17: 9476 / 10000
Epoch 18: 9483 / 10000
Epoch 19: 9484 / 10000
Epoch 20: 9489 / 10000
Epoch 21: 9497 / 10000
Epoch 22: 9481 / 10000
Epoch 23: 9506 / 10000
Epoch 24: 9501 / 10000
Epoch 25: 9494 / 10000
Epoch 26: 9491 / 10000
Epoch 27: 9480 / 10000
Epoch 28: 9485 / 10000
Epoch 29: 9492 / 10000


Just using basic Matrices we were able to build a classifier with such high confidence of `94.9%`